In [1]:
import IJulia
import Base64

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 5.5
fig_height = 3.5
fig_format = :pdf
fig_dpi = 300

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = "L2hvbWUvdml0dXJpL0RvY3VtZW50b3MvR2l0SHViL2Jsb2cvcG9zdHMtZHMvanVsaWEtZm9yLXItdXNlcnM="
  if !isempty(run_path)
    run_path = String(Base64.base64decode(run_path))
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| eval: false

@edit maximum([1:5;])

In [3]:
#julia
# define power2 for numbers
power2(x) = x^2;

# apply in vectors
power2.(1:10)

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [4]:
#julia
X = reshape([1:16;], (4, 4))

4×4 Matrix{Int64}:
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16

In [5]:
#julia
power2.(X)

4×4 Matrix{Int64}:
  1  25   81  169
  4  36  100  196
  9  49  121  225
 16  64  144  256

In [6]:
#julia
[1:5;] .+ 10

5-element Vector{Int64}:
 11
 12
 13
 14
 15

In [7]:
#| include: false
# import Pkg; Pkg.add("RCall"); Pkg.add("BenchmarkTools");
using RCall;

In [8]:
#| warning: false
#R
R"""
f1_vec = function(x) {
    y = x + 1
}

f1_loop = function(x) {
    y = x
    for (i in seq_along(x)) y[i] = x[i] + 1
    y
}

f1_map = function(x) {
    purrr::map_dbl(x, \(xi) xi + 1)
}

x = 1:100000

bench::mark(
    f1_vec(x)
    ,f1_loop(x)
    ,f1_map(x)
    ,relative = TRUE
)
"""

┌ Warning: RCall.jl: Aviso: Some expressions had a GC in every iteration; so filtering is disabled.
└ @ RCall ~/.julia/packages/RCall/0ggIQ/src/io.jl:172


RObject{VecSxp}
# A tibble: 3 × 13
  expression   min median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc total_time
  <bch:expr> <dbl>  <dbl>     <dbl>     <dbl>    <dbl> <int> <dbl>   <bch:tm>
1 f1_vec(x)    1      1       356.       1.50    41.3   2469    41      500ms
2 f1_loop(x)  45.2   52.7      10.2      1.50     1       71     1      503ms
3 f1_map(x)  544.   529.        1        1        8.97     7     9      505ms
# ℹ 4 more variables: result <list>, memory <list>, time <list>, gc <list>


In [9]:
#julia
f1_vec(x) = x .+ 1;

function f1_loop(x)
    y = similar(x)
    @inbounds for i ∈ eachindex(x) y[i] = x[i] + 1 end
    y
end;

function f1_map(x)
    map(x) do xi
        xi + 1 
    end
end;

In [10]:
#| eval: false
#julia
using BenchmarkTools;
x = [1:100000;];

@benchmark f1_vec($x)

In [11]:
#| eval: false
#julia
@benchmark f1_loop($x)

In [12]:
#| eval: false
#julia
@benchmark f1_map($x)

In [13]:
#| eval: false

using MyPackage.Metrics

# or
import MyPackage.Models as MD